In [24]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
import os
from sklearn.metrics.pairwise import cosine_similarity


In [10]:
data = list()
listdir = os.listdir('texts/')

In [11]:
i = 0
for file_name in listdir:
    text_path = f'texts/{file_name}'
    with open(text_path, 'r') as f:
        text = f.read()
    data.append([file_name, text])
    if i>11:
        break
    i+=1

In [12]:
df = pd.DataFrame(data, columns=['file', 'text'])

In [13]:
df

,file,text
0,170351.txt,"Santa Claus Obesity: Santa Is Getting Fatter, ..."
1,10457.txt,Dalai Lama Meets 'Old Trusted Friend' Barack O...
2,137111.txt,Google Glass: Qué Guay!\nRewind 24 hours as I ...
3,120597.txt,"Teen Impregnation Is the Real Problem, Not Tee..."
4,21665.txt,Donald Trump's Lawyer Claims President Was Nev...
5,185480.txt,Weekly Roundup of eBay Vintage Clothing Finds ...
6,70265.txt,New Photos Of Possible 'Gilmore Girls' Set Sur...
7,146488.txt,The Sounds of Saturn and Other Space Oddities\...
8,102344.txt,A Diet For Food Lovers\n
9,156028.txt,While Everyone Is Looking\nEvery generation ha...


In [14]:
df = df.head(10)
df

,file,text
0,170351.txt,"Santa Claus Obesity: Santa Is Getting Fatter, ..."
1,10457.txt,Dalai Lama Meets 'Old Trusted Friend' Barack O...
2,137111.txt,Google Glass: Qué Guay!\nRewind 24 hours as I ...
3,120597.txt,"Teen Impregnation Is the Real Problem, Not Tee..."
4,21665.txt,Donald Trump's Lawyer Claims President Was Nev...
5,185480.txt,Weekly Roundup of eBay Vintage Clothing Finds ...
6,70265.txt,New Photos Of Possible 'Gilmore Girls' Set Sur...
7,146488.txt,The Sounds of Saturn and Other Space Oddities\...
8,102344.txt,A Diet For Food Lovers\n
9,156028.txt,While Everyone Is Looking\nEvery generation ha...


In [15]:
# df.to_csv('text_df.csv')

In [16]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [17]:
tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [18]:
tokenized.shape

(10,)

In [19]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [20]:
padded.shape

(10, 70)

In [21]:
input_ids = torch.tensor(np.array(padded))

In [22]:
with torch.no_grad():
    last_hidden_states = model(input_ids)

In [23]:
features = last_hidden_states[0][:,0,:].numpy()
features.shape

(10, 768)

In [39]:
arr = cosine_similarity([features[0]], features[1:])[0]
arr

array([0.8729771 , 0.9065465 , 0.8159222 , 0.91213655, 0.92439944,
       0.92070615, 0.94265056, 0.8820535 , 0.8727509 ], dtype=float32)

In [42]:
res  = arr.argsort()[-10:][::-1]

In [44]:
[df.file[i] for i in res]

['70265.txt',
 '21665.txt',
 '185480.txt',
 '120597.txt',
 '10457.txt',
 '146488.txt',
 '170351.txt',
 '102344.txt',
 '137111.txt']